# More Tables
So far, we have looked at basic ways to read, create, and view tables. In this lecture, we are going to learn how to sort columns, select rows with even greater precision, join tables together, and summarize columns.

In [ ]:
# This cell needs to be run first; don't worry about why just yet!
# Click on the cell to highlight it, then press Shift+Enter or Control+Enter to run it.
from datascience import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
%matplotlib inline

## Exploring TIMIT Data <a id='timit'></a>

Our corpus for this week is [TIMIT](https://en.wikipedia.org/wiki/TIMIT), a database of speech recorded during telephone conversations in eight different varieties of American English. This database is specifically designed for phonetics and speech recognition research. For more information, visit [their website](https://catalog.ldc.upenn.edu/LDC93S1) (Garofolo et al. 1993).

First, we will upload the `.csv` file called `wk2-timit.csv`, which is located in the same folder as this notebook.

In [ ]:
timit = Table.read_table('wk2-timit.csv')

As before, we can use the method `.show()` to show just the first few rows of the table.

In [ ]:
timit.show(5)

Look at the table you've created and try to figure out what each column measures as well as its type. Each column represents a different attribute.

|Column Name|Details|
|---|---|
|speaker|unique speaker ID|
|gender|Speaker’s self-reported gender|
|region|Speaker dialect region number|
|word|Lexical item (from sentence prompt)|
|vowel|Vowel ID|
|duration|Vowel duration (seconds)|
|F1/F2/F3/f0|f0 and F1-F3 in BPM (Hz)|

Sometimes data is encoded with with an identifier, or key, to save space and simplify calculations. Each of those keys corresponds to a specific value. If you look at the `region` column, you will notice that all of the values are numbers. Each of those numbers corresponds to a region, for example, in our first row the speaker, `cjf0`, is from region `1`. That corresponds to New England. Below is a table with all of the keys for `region`.

|Key|Region|
|---|---|
|1|New England|
|2|Northern|
|3|North Midland|
|4|South Midland|
|5|Southern|
|6|New York City|
|7|Western|
|8|Army Brat|

## Sorting columns
If you'd like to reorganize the data in your table, you can use the method `.sort()`. The argument it takes is the name of the column (as a string). By default, this method sorts a column in ascending order. For example, we can sort `timit` so that the vowels with the shortest duration are up top.

In [ ]:
timit.sort('duration').show(10)

You can reverse this by adding an argument to the method: `descending=True`. The line below will show the same data, but with the longest vowels up top.

In [ ]:
timit.sort('duration', descending=True).show(10)

The `sort` method can take one more argument, `distinct`. If set to `True`, it returns only unique values. At times, this may be a useful technique to get the unique (or distinct) values from a particular column. The line below sorts `timit` by the height of the speakers, but only returns distinct (or unique) values for `height`.

In [ ]:
timit.sort('height', distinct=True)

Adding the attribute `.num_rows`, then, can tell us how many rows are in the table that only shows unique heights.

In [ ]:
timit.sort('height', distinct=True).num_rows

This shows us that there are 79 unique values for `height` in the entire TIMIT database. How many unique words are in the database?

In [ ]:
timit.sort('word', distinct=True).num_rows

Remember from HW1 that you can use the function `np.unique()` from the `numpy` package to do the same thing. First, create an array from the table using the `.column()` method.

In [ ]:
words = timit.column('word')
words

Then we put that array into the function `np.unique()`.

In [ ]:
np.unique(words)

And finally, use the function `len()` to determine the length of this array.

In [ ]:
len(np.unique(words))

If you're curious about creating a list of all of these unique words, use the method `.tolist()` and the function `print()`. Notice that all of the words are in all caps. Python is case sensitive, so if you want to refer to any of the words in this list/array, remember to use all caps.

In [ ]:
print(np.unique(words).tolist())

## Selecting rows that satisfy a condition (revisited)
So far, we only have learned how to select observations in a table which exactly equal a certain value in a certain column by using the method `.where(column_name, value)`

In [ ]:
ow = timit.where('vowel', 'OW')
ow.show(5)

Other filtering/selection operations can be done using predicates with the special object `are` (which is unique and rather special to the `datascience` package). For example, "greater than 5" would be `are.above(5)`. Remember, these are used as the `value` argument in `.where(column_name, value)`. In the code below, we are selecting rows in `timit` "where" the value for the column `height` is greater than `180`. Be careful with the distinction between strings and integers here!

In [ ]:
timit.where('height', are.above(180))

To find all of the rows that are between certain values, use `are.between(x, y)`:

In [ ]:
timit.where('duration', are.between(0.05, 0.09))

There are eight predicates that can be used for numerical filtering:

1. `are.equal_to(x)`: (identical to just `x`)
2. `are.above(x)`: Greater than x
3. `are.above_or_equal_to(x)`: Greater than or equal to x
4. `are.below(x)`: Less than x
5. `are.below_or_equal_to(x)`: Less than or equal to x
6. `are.between(x, y)`: Greater than or equal to x, and less than y
7. `are.strictly_between(x, y)`: Greater than x and less than y
8. `are.between_or_equal_to(x, y)`: Greater than or equal to x, and less than or equal to y

In addition, there is one predicate which can be used to filter based on strings: `are.containing(S)`. This filters a table based on whether or not the string `S` is contained within the row. For instance, we could filter based on whether or not the word contains the letter "Z". Don't forget to use the capital letter, since the strings for all the words in the database are in all caps.

In [ ]:
timit.where('word', are.containing('Z'))

What do you think will happen if you run the line below?

In [ ]:
timit.where('word', are.containing('z'))

Negatives exist for many of the predicates... For instance, `are.not_equal_to`:

In [ ]:
timit.where('gender', are.not_equal_to('male'))

## Joining two tables by a column
Often, you will have two tables with related information that you will need to combine. For example, you have the vowel information as ARPABET symbols, but you may want the vowels in the IPA (International Phonetic Alphabet).

The first cell below creates a Table that has ARPABET vowel symbols in one column and their corresponding IPA symbols in another column.

In [ ]:
np.unique(timit.column('vowel')) # quick double-check of the unique values in the 'vowel' column
ipa = Table().with_columns(
    'vowel', make_array('AA','AE','AH','AO','EH','ER','EY','IH','IY','OW','UH','UW'),
    'vowel_IPA', make_array('a','æ','ʌ','ɔ','ɛ','ɚ','eɪ','ɪ','i','oʊ','ʊ','u'))
ipa

This can be added to the original `timit` table using the method `t1.join(column_name, t2)`, where the first table `t1` is operated on, and the two arguments are the column name they have in common and the second table `t2`. Note that unlike arrays, your tables do not have to have the same number of rows or columns to be joined in this way. The method looks for the common column, and it will repeat values if necessary to "fill in" the rest of the table.

In [ ]:
timit.join('vowel', ipa)

What if the column names do not match? The two tables can still be joined by adding a third argument: `t1.join(t1_column_name, t2, t2_column_name)`:

In [ ]:
ipa2 = Table().with_columns(
    'vowel_ARP', make_array('AA','AE','AH','AO','EH','ER','EY','IH','IY','OW','UH','UW'),
    'vowel_IPA', make_array('a','æ','ʌ','ɔ','ɛ','ɚ','eɪ','ɪ','i','oʊ','ʊ','u'))

In [ ]:
timit.join('vowel', ipa2, 'vowel_ARP')

Note that the resulting table uses the column name from `t1`. Reversing `t1` and `t2` in `t1.join(t1_column_name, t2, t2_column_name)` changes the resulting column label. However, the data in the two tables are identical.

In [ ]:
ipa2.join('vowel_ARP', timit, 'vowel')

Anyway, we want to work with the version of the table that includes IPA from here on out, so let's assign this joined table to a new variable.

In [ ]:
t = timit.join('vowel', ipa)

## Summarizing columns

With so many observations (rows) in our table, it would be helpful to have some summary statistics of each column. For example, we may be interested in the f0 (fundamental frequency, which is roughly equivalent to vocal pitch) of our speakers. It is generally assumed that (cisgender) women have higher-pitched voices than (cisgender) men. Do the data in TIMIT support this?

First, let's split our `timit` table into two smaller tables by gender. To identify the possible values of gender in the `gender` column, we can use the function `np.unique()` on the column.

In [ ]:
np.unique(timit.column('gender'))

Okay, only two genders in this Table. Given that information, we'll split `timit` into two smaller tables, one for females (`t_female`) and one for males (`t_male`). Creating these subset dataframes does not affect the original table `timit`.

In [ ]:
t_female = t.where('gender','female')
t_male = t.where('gender','male')

The function that calculates mean is `np.mean()`. Its argument is an array, which means that we need to give it `t_female.column()`. (Remember that the `.column()` method returns an array.)

In [ ]:
np.mean(t_female.column('f0'))

This tells us that the mean (or average) value for f0 in the `t_female` table is about 203 Hertz. How about in `t_male`?

In [ ]:
f = np.mean(t_female.column('f0'))
m = np.mean(t_male.column('f0'))
print("The average f0 for female speakers in TIMIT is",f,
      "Hertz, and the average f0 for male speakers in TIMIT is",m,"Hertz.")

(You may have noticed different ways for incorporating numeric objects (integers and floats) into strings for printing in this lecture compared to last week's lecture. If you are curious about the differences, feel free to ask about them in the Live Discussion!)

Finally, other useful functions from `numpy` for calculating summary statistics include `np.median()`, `np.min()`, `np.max()`, `np.std()`, and `np.sum()`. In fact, there are many functions in the library, and Jupyter notebooks allow for a neat trick called `tab completion`. If you begin typing in the name of a method or function and then press `Tab` on your keyboard, various autocomplete options will come up, and you can select one of them. Try it below:

In [ ]:
np.

In [ ]:
print("The median male f0 value is",np.median(t_male.column('f0')),"Hertz.")
print("The standard deviation for male f0 is",np.std(t_male.column('f0')),"Hertz.")
print("The male with the lowest f0 value had an f0 of",np.min(t_male.column('f0')),
      "Hertz, which... yikes, that is most definitely a measurement error!")

Okay, that's all for now! In the next lecture, we will start learning basic visualizations for data, using the TIMIT database again.